In [5]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split


In [6]:
class ImageClassifier(pl.LightningModule):
    def __init__(self, model, learning_rate=0.001):
        super(ImageClassifier, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        self.log('val_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [7]:
# Data Loader
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset_path = "seg_train"
dataset = ImageFolder(dataset_path, transform=transform)
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64)


In [ ]:

# Model setup
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)  # 6 classes in the dataset

In [ ]:

# Lightning Module and Trainer setup
model = ImageClassifier(model)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='/content/checkpoints',
    filename='best_model'
)

In [ ]:

trainer = pl.Trainer(
    max_epochs=10,
    # gpus=0,
    callbacks=[checkpoint_callback],
    # progress_bar_refresh_rate=30
)

In [ ]:
# Training
trainer.fit(model, train_loader, test_loader)

In [22]:
correct = 0
total = 0
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient tracking
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy:.2f}")

Accuracy on the test set: 0.90
